In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')
import random
import pandas as pd
import numpy as np
import os
import glob

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore') 

In [4]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [5]:
device

device(type='cuda')

## load data

In [6]:
all_input_list = sorted(glob.glob('./train_input/*.csv'))
all_target_list = sorted(glob.glob('./train_target/*.csv'))

In [7]:
train_input_list = all_input_list[:25]#25개만 train으로 사용
train_target_list = all_target_list[:25]

val_input_list = all_input_list[25:]
val_target_list = all_target_list[25:]

In [10]:
a=pd.read_csv(train_input_list[0])
a.head()

,DAT,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,일간누적분무량,시간당백색광량,일간누적백색광량,시간당적색광량,일간누적적색광량,시간당청색광량,일간누적청색광량,시간당총광량,일간누적총광량
0,0,00:00,25.300000,81.835000,536.016667,1.407439,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0
1,0,01:00,25.680357,81.264286,528.696429,1.409003,126.0,126.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0
2,0,02:00,25.273333,81.471666,532.833333,1.406913,0.0,126.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0
3,0,03:00,25.355000,81.398334,545.566667,1.406689,126.0,252.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0
4,0,04:00,25.391667,81.483333,558.583333,1.411070,0.0,252.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0


In [11]:
b=pd.read_csv(train_input_list[0])
b.head()

,DAT,predicted_weight_g
0,1,0.167719
1,2,0.181787
2,3,0.265921
3,4,0.423650
4,5,0.475272


In [35]:
df = pd.DataFrame()
for i in range(len(train_input_list)):
    data=pd.read_csv(train_input_list[i])
    df = pd.concat([df,data])

In [36]:
df

,DAT,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,일간누적분무량,시간당백색광량,일간누적백색광량,시간당적색광량,일간누적적색광량,시간당청색광량,일간누적청색광량,시간당총광량,일간누적총광량
0,0,00:00,25.300000,81.835000,536.016667,1.407439,0.00,0.00,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0,0.0000
1,0,01:00,25.680357,81.264286,528.696429,1.409003,126.00,126.00,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0,0.0000
2,0,02:00,25.273333,81.471666,532.833333,1.406913,0.00,126.00,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0,0.0000
3,0,03:00,25.355000,81.398334,545.566667,1.406689,126.00,252.00,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0,0.0000
4,0,04:00,25.391667,81.483333,558.583333,1.411070,0.00,252.00,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,27,19:00,25.761667,59.603333,557.000000,0.951505,0.00,3350.56,0.0,104623.8974,0.0,33509.7,0.0,21155.5825,0.0,159289.1799
668,27,20:00,26.860000,59.385000,559.850000,0.947002,126.00,3476.56,0.0,104623.8974,0.0,33509.7,0.0,21155.5825,0.0,159289.1799
669,27,21:00,27.395000,58.901667,552.183333,0.943065,0.00,3476.56,0.0,104623.8974,0.0,33509.7,0.0,21155.5825,0.0,159289.1799
670,27,22:00,27.940000,58.513333,546.516667,0.939832,119.57,3596.13,0.0,104623.8974,0.0,33509.7,0.0,21155.5825,0.0,159289.1799


In [37]:
df.columns

Index(['DAT', 'obs_time', '내부온도관측치', '내부습도관측치', 'co2관측치', 'ec관측치', '시간당분무량',
       '일간누적분무량', '시간당백색광량', '일간누적백색광량', '시간당적색광량', '일간누적적색광량', '시간당청색광량',
       '일간누적청색광량', '시간당총광량', '일간누적총광량'],
      dtype='object')

In [38]:
df.drop('obs_time',axis = 'columns')

,DAT,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,일간누적분무량,시간당백색광량,일간누적백색광량,시간당적색광량,일간누적적색광량,시간당청색광량,일간누적청색광량,시간당총광량,일간누적총광량
0,0,25.300000,81.835000,536.016667,1.407439,0.00,0.00,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0,0.0000
1,0,25.680357,81.264286,528.696429,1.409003,126.00,126.00,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0,0.0000
2,0,25.273333,81.471666,532.833333,1.406913,0.00,126.00,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0,0.0000
3,0,25.355000,81.398334,545.566667,1.406689,126.00,252.00,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0,0.0000
4,0,25.391667,81.483333,558.583333,1.411070,0.00,252.00,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,27,25.761667,59.603333,557.000000,0.951505,0.00,3350.56,0.0,104623.8974,0.0,33509.7,0.0,21155.5825,0.0,159289.1799
668,27,26.860000,59.385000,559.850000,0.947002,126.00,3476.56,0.0,104623.8974,0.0,33509.7,0.0,21155.5825,0.0,159289.1799
669,27,27.395000,58.901667,552.183333,0.943065,0.00,3476.56,0.0,104623.8974,0.0,33509.7,0.0,21155.5825,0.0,159289.1799
670,27,27.940000,58.513333,546.516667,0.939832,119.57,3596.13,0.0,104623.8974,0.0,33509.7,0.0,21155.5825,0.0,159289.1799


In [39]:
train=df

In [40]:
train

,DAT,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,일간누적분무량,시간당백색광량,일간누적백색광량,시간당적색광량,일간누적적색광량,시간당청색광량,일간누적청색광량,시간당총광량,일간누적총광량
0,0,00:00,25.300000,81.835000,536.016667,1.407439,0.00,0.00,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0,0.0000
1,0,01:00,25.680357,81.264286,528.696429,1.409003,126.00,126.00,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0,0.0000
2,0,02:00,25.273333,81.471666,532.833333,1.406913,0.00,126.00,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0,0.0000
3,0,03:00,25.355000,81.398334,545.566667,1.406689,126.00,252.00,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0,0.0000
4,0,04:00,25.391667,81.483333,558.583333,1.411070,0.00,252.00,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,27,19:00,25.761667,59.603333,557.000000,0.951505,0.00,3350.56,0.0,104623.8974,0.0,33509.7,0.0,21155.5825,0.0,159289.1799
668,27,20:00,26.860000,59.385000,559.850000,0.947002,126.00,3476.56,0.0,104623.8974,0.0,33509.7,0.0,21155.5825,0.0,159289.1799
669,27,21:00,27.395000,58.901667,552.183333,0.943065,0.00,3476.56,0.0,104623.8974,0.0,33509.7,0.0,21155.5825,0.0,159289.1799
670,27,22:00,27.940000,58.513333,546.516667,0.939832,119.57,3596.13,0.0,104623.8974,0.0,33509.7,0.0,21155.5825,0.0,159289.1799


In [41]:
dataFeatures = []
dataType     = []
null         = []
nullPCT      = []
unique       = []
minValue     = []
maxValue     = []
uniqueSample = []


for item in list(train):
    dataFeatures.append(item)

for item in dataFeatures:
    dataType.append(train[item].dtype.name)
    
for item in dataFeatures:
    null.append(len(train[train[item].isnull() == True]))

for item in dataFeatures:
    nullPCT.append(round(len(train[train[item]== 0])/len(train[item])*100,2))
    
for item in dataFeatures:
    minValue.append(train[item].min())

for item in dataFeatures:
    maxValue.append(train[item].max())

for item in dataFeatures:
    unique.append(train[item].nunique())

for item in dataFeatures:
    uniqueSample.append(train[item].unique()[0:2])

train_info = pd.DataFrame({
    'dataFeatures' : dataFeatures,
    'dataType' : dataType,
    'null' : null,
    'nullPCT':nullPCT,
    'unique' : unique,
    'minValue' : minValue,
    'maxValue' : maxValue,
    'uniqueSample':uniqueSample
})
train_info

,dataFeatures,dataType,null,nullPCT,unique,minValue,maxValue,uniqueSample
0,DAT,int64,0,3.57,28,0,27,"[0, 1]"
1,obs_time,object,0,0.00,70,00:00,23:00:00.,"[00:00, 01:00]"
2,내부온도관측치,float64,0,0.01,15571,0.0,40.363333,"[25.30000013, 25.68035718]"
3,내부습도관측치,float64,0,0.01,16068,0.0,93.350002,"[81.83500023, 81.26428591]"
4,co2관측치,float64,0,0.00,12927,314.883333,1714.85,"[536.0166667, 528.6964286]"
5,ec관측치,float64,0,12.33,14321,0.0,5.462324,"[1.407439293, 1.409002968]"
6,시간당분무량,float64,0,25.57,1818,-653.26,3144.13,"[0.0, 126.0]"
7,일간누적분무량,float64,0,2.76,5476,0.0,42855.23,"[0.0, 126.0]"
8,시간당백색광량,float64,0,57.85,1041,-138371.2461,18564.6,"[0.0, 17636.37]"
9,일간누적백색광량,float64,0,45.89,3614,0.0,292014.9698,"[0.0, 17636.37]"


In [42]:
percent20UP = list(train_info[train_info['nullPCT'] >= 20.0]['dataFeatures'])

In [44]:
percent20UP

['시간당분무량',
 '시간당백색광량',
 '일간누적백색광량',
 '시간당적색광량',
 '일간누적적색광량',
 '시간당청색광량',
 '일간누적청색광량',
 '시간당총광량',
 '일간누적총광량']

In [45]:
train = train.drop(percent20UP,axis=1)

In [46]:
train

,DAT,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,일간누적분무량
0,0,00:00,25.300000,81.835000,536.016667,1.407439,0.00
1,0,01:00,25.680357,81.264286,528.696429,1.409003,126.00
2,0,02:00,25.273333,81.471666,532.833333,1.406913,126.00
3,0,03:00,25.355000,81.398334,545.566667,1.406689,252.00
4,0,04:00,25.391667,81.483333,558.583333,1.411070,252.00
...,...,...,...,...,...,...,...
667,27,19:00,25.761667,59.603333,557.000000,0.951505,3350.56
668,27,20:00,26.860000,59.385000,559.850000,0.947002,3476.56
669,27,21:00,27.395000,58.901667,552.183333,0.943065,3476.56
670,27,22:00,27.940000,58.513333,546.516667,0.939832,3596.13
